In [1]:
%run python_help.py

In [2]:
file_name = "./data/fl_500_7"
input_data = open(file_name).read()
facilities,customers = read_data(input_data)

customer_cost = gen_customer_cost(facilities,customers)
customer_damend = pd.Series(list(map(lambda x:x.demand,customers ))).astype("int32")
facility_cost = pd.Series(list(map(lambda x:x.setup_cost,facilities))).astype("int32")
facility_capacity = pd.Series(list(map(lambda x:x.capacity,facilities))).astype("int32")
facility_size = len(facilities)
customer_size = len(customers)
facility_min_size = gen_min_facility_size(facilities,customers)
facility_min_size

172

In [3]:
def takeVar(l):return list(filter(lambda x:type is Variable,l))

class MyModel(ModelHelp):
    def __init__(self,model,fs_vtype,cs_vtype):
        
        fs = facility_cost.map(lambda x: model.addVar(vtype=fs_vtype,lb=0,ub=1))
        cs = customer_cost.applymap(lambda x: model.addVar(vtype=cs_vtype,lb=0,ub=1))
        
        objective = quicksum(fs * facility_cost) + quicksum(cs.values.reshape(-1) * customer_cost.astype("int32").values.reshape(-1))

        for index,row in cs.iterrows():
            model.addCons(quicksum(row) == 1)
            
        for f,fc,c in zip(fs,facility_capacity,cs):
            damend = quicksum(cs[c] * customer_damend)
            model.addCons(damend <= f * fc )
            
        model.setObjective(objective)
        self.model = model 
        self.objective = objective
        self.fs = fs
        self.cs = cs
        self.facilities = facilities
        self.customers = customers

In [4]:
lr_model = MyModel(Model("lr-facility"),fs_vtype="C",cs_vtype="B")
lr_model.set_min_facility(facility_min_size)
lr_model.set_limit_time(1500)
lr_model.optimize()

{'status': 'optimal',
 'time': 656.0,
 'total_time': 685.0,
 'obj ': 23874876.383960444,
 'cost': 37099453.174958944}

In [5]:
fs_value = lr_model.fs_value()
print(lr_model.fs_one())
print(lr_model.fs_zero())

Int64Index([], dtype='int64')
Int64Index([  1,   2,  20,  28,  59,  71,  79,  80,  83, 101, 102, 122, 126,
            174, 199, 204, 212, 245, 262, 281, 286, 291, 303, 304, 315, 328,
            336, 341, 358, 389, 396, 417, 426, 435, 439, 447, 459, 461, 463,
            466, 476, 491, 492, 499],
           dtype='int64')


In [6]:
lr_model.model.writeBestSol(f"{file_name}.best_sol")
# lr_model.model.readSol(f"{file_name}.best_sol")

In [7]:
import gc
del lr_model
gc.collect()


6002000

In [8]:
# for i in range(facility_min_size,min(facility_min_size + 10,len(facilities))):
#     mip_model = MyModel(Model("lr-facility"),fs_vtype="B",cs_vtype="B")
#     mip_model.set_facility_size(i)
#     print(i,mip_model.optimize())


In [4]:
mip_model = MyModel(Model("mip-facility"),fs_vtype="B",cs_vtype="B")
for value in mip_model.fs[[ 1,   2,  20,  28,  59,  71,  79,  80,  83, 101, 102, 122, 126,
            174, 199, 204, 212, 245, 262, 281, 286, 291, 303, 304, 315, 328,
            336, 341, 358, 389, 396, 417, 426, 435, 439, 447, 459, 461, 463,
            466, 476, 491, 492, 499]]:
    mip_model.model.fixVar(value,0)
# for value in mip_model.fs[fs_value <= 0.0 ]:
#     mip_model.model.fixVar(value,0)
# for value in mip_model.fs[fs_value >= 1 ]:
#     mip_model.model.fixVar(value,1)

# mip_model.set_facility_size(facility_min_size)

# mip_model.model.setRealParam("limits/gap",0.05)
mip_model.set_min_facility(facility_min_size)
gc.collect()

NameError: name 'gc' is not defined

In [7]:
mip_model.set_limit_time((10+60)*60)
mip_model.optimize()

{'status': 'timelimit',
 'time': 4229.0,
 'total_time': 4569.0,
 'obj ': 31737568.99999999,
 'cost': 31739247.213276297}

In [6]:
mip_model.model.writeBestSol(f"{file_name}.mip_best_sol")


In [12]:
solution = mip_model.solution
used = mip_model.used
cost = mip_model.cost
print(len(set(solution)))
print(mip_model.cost)

340
32860836.071739372


In [ ]:
file = open(f"{file_name}.result","w")
output_data = '%.2f' % cost + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, solution))
file.write(output_data)
file.close()

In [ ]:
[ 0,  2,  3,  4,  6, 13, 14, 18, 25, 26, 38, 42, 43, 44, 45, 55, 65,
            71, 75, 77, 81, 86, 95, 97, 98]
print(mip_model.fs_one())
print(mip_model.fs_zero())